In [1]:
import import_ipynb
import CA
import pprint
CAPubKeyC = []
reqFrame = {}
clientName = 'Tester'
role = 2
ClientCert = []
RecCert = []
clientPubKey = []
clientPriKey = []

importing Jupyter notebook from CA.ipynb


In [2]:
def CertificateRoutine():
    global ClientCert, clientPubKey, clientPriKey, CAPubKeyC, reqFrame, clientName, role
    import socket
    import pickle
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect(('127.0.0.7', 8080))
    global ClientCert
    if CAPubKeyC == []:
        reqFrame = {"req":1} # requesting for CA's PublicKey
        data = pickle.dumps(reqFrame)
        print("Requesting for CA's PublicKey \t:\t")
        pprint.pprint(data)
        print('\n')
        client.send(data)
        data = client.recv(4096)
        CAPubKeyC = pickle.loads(data)
        print("CA's PublicKey \t:\t")
        pprint.pprint(data)
        print('\n')
    if ClientCert == []: #Generating Subject'S Public,Private Key pair
        if clientPubKey == []:
            clientPubKey , clientPriKey = CA.KeyGen()
        reqFrame = {"req" : 2, "subName" : clientName, "role":role, "pubKey" : clientPubKey} #Request for certificate
        data = pickle.dumps(reqFrame)
        print("Request for Certificate Generation \t:\t")
        pprint.pprint(data)
        print('\n')
        client.send(data)
        data = client.recv(4096)
        fromServer = pickle.loads(data)
#         print(fromServer)
        ClientCert = CA.certificate()
        ClientCert.dict2cert(fromServer)
        print("Generated Certificate \t:\t")
        pprint.pprint(ClientCert.transmitCertificate())
        print('\n')
    client.close()
CertificateRoutine()

Requesting for CA's PublicKey 	:	
b'\x80\x04\x95\x0c\x00\x00\x00\x00\x00\x00\x00}\x94\x8c\x03req\x94K\x01s.'


CA's PublicKey 	:	
(b'\x80\x04\x95g\x00\x00\x00\x00\x00\x00\x00\x8c\x07rsa.key\x94\x8c\tPublicKey'
 b'\x94\x93\x94)\x81\x94\x8aA\x89f\xb7\xee\xd5\x80{z.\xf9\xa9\xd1\xb8~\xa1\xad'
 b'\xceh\xd5\x15>~0+$3\xe6\xb6\xf6\xfc\xb0(c\xf3\t\x8f\xfbH\x8a\xd0x\xae?/'
 b'\xd40K\xc0\xf9U\xd6e\x8d,\xee\xbb\xfeU\xfe\x85v\x8c\x00\x8c\x00J\x01\x00'
 b'\x01\x00\x86\x94b.')


Request for Certificate Generation 	:	
(b'\x80\x04\x95\x98\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\x03req\x94'
 b'K\x02\x8c\x07subName\x94\x8c\x06Tester\x94\x8c\x04role\x94K\x02\x8c\x06pubK'
 b'ey\x94\x8c\x07rsa.key\x94\x8c\tPublicKey\x94\x93\x94)\x81\x94\x8aA'
 b'\x157\xb0\xd0\xbc\x10\x11lX\x88\x9f=7\xe4\xdaeP\xdfWjz\x9f\x14\xc7\xfc\xceww'
 b'\x91\x8c\t={\xd7\x03P\xde\xab\x1a\x05\xfb\xbfy\x80\xc4M\x14\x81@\x0b\x9e#'
 b'\x8eR\xf9\x91\x10T\x89\x89\xc7\xedH\x91\x00J\x01\x00\x01\x00\x86\x94bu.')


Generated Certificate 	:	
{'ext':

In [277]:

import numpy as np
def RandomGenerator():
    random_bytes = str(np.random.bytes(10).hex())
    seed = random_bytes
#     for i in range(0,2):
#         random_bytes = str(np.random.bytes(1).hex())
#         seed = seed +' '+ random_bytes
    return seed.upper()
def Req(data):
    global ClientCert, clientPriKey
    if data["sFn"] == '08':
        if data['Cnf'] == '01':
            print("Authentication using Challenge Response Supported")
        elif data['Cnf'] == '02':
            print("Authentication using PKI Certificate Exchange Supported ")
            from_client = {}
            from_client["Fn"] = '29'
#             sf = int(input("Enter Requested subFunction \n 1 : verifyCertificateUnidirectionally \n 2: verifyCertificateBidirectionally "))
#             print(sf)
            sf = 2
            if sf == 2:
                from_client["sFn"] = "02" 
            else:
                from_client["sFn"] = "01"
            print(from_client["sFn"])
            from_client["CC"] = '00'
            from_client["Cert"] = ClientCert
            if from_client["sFn"] =='02':
                from_client["CCli"] = RandomGenerator()
            return from_client
    elif data["sFn"] == '01':
        print("Unidirectional Authetication")
        temp = CA.sign(data["CS"],clientPriKey)
        from_client = {}
        from_client["Fn"] = '29'
        from_client["sFn"] = '03'
        from_client["POO"] = temp
        return from_client
    elif data["sFn"] == '02':
        print("Bidirectional Authentication")
        temp = CA.sign(data["CS"],clientPriKey)
        from_client = {}
        from_client["Fn"] = '29'
        from_client["sFn"] = '03'
        from_client["POO"] = temp
        return from_client
            
            
# CA.verifyCertificate(ClientCert,CAPubKeyC )

In [173]:
timeanalysis= pd.DataFrame([])

In [275]:
import datetime
import socket
import pickle
import pandas as pd
from datetime import datetime

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect(("127.0.0.1", 9090))
for _ in range(0,100): 
    start = datetime.now()
    reqFrame = {"Fn":'29','sFn': '08'}
    from_client = {}
    data = pickle.dumps(reqFrame)
    client.send(data)
    data = client.recv(4096)
    ChaCli=''
    from_client = pickle.loads(data)
    if from_client["Cnf"] == '02' and from_client["Fn"] == '69':
        tempData = Req(from_client)
        from_client = tempData
        if from_client['sFn'] == '02':
            ChaCli = from_client['CCli']
        data = pickle.dumps(from_client)
        client.send(data)
        data = client.recv(4096)
        from_client = pickle.loads(data)
        
    if from_client["Cnf"] == '11' and from_client["Fn"] == '69':
        if from_client['sFn'] == '02':
            RecCert = from_client['ECer']
            if CA.verifyCertificate(RecCert,CAPubKeyC ):
                print("Server Certificate Verified")
                if CA.verifySign(ChaCli,from_client["POOC"],RecCert.subjectPublicKey):
                    print("Certificate Verified with proof of Ownership")
    #     temp = CA.sign(from_client["CS"],clientPriKey)
        tempData = Req(from_client)
        from_client = tempData
        data = pickle.dumps(from_client)
        client.send(data)
        data = client.recv(4096)
        from_client = pickle.loads(data)
    if from_client['Cnf'] == "12" and from_client["Fn"] == '69':
        print("Authentication Completed")
    client.close()
    end = datetime.now()
    elapsetime = (end - start).total_seconds()
    timeanalysis = timeanalysis.append([elapsetime])

Authentication using PKI Certificate Exchange Supported 
01
Unidirectional Authetication
Authentication Completed


C:\Users\A477967\AppData\Local\Temp/ipykernel_19864/456224696.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  timeanalysis = timeanalysis.append([elapsetime])


OSError: [WinError 10038] An operation was attempted on something that is not a socket

In [276]:
print('maximum number of iteration',timeanalysis.values.max())
print('minimum number of iteration',timeanalysis.values.min()) 
print('Average number of iteration', timeanalysis.values.mean())

maximum number of iteration 0.007002
minimum number of iteration 0.002996
Average number of iteration 0.00475777


In [287]:
timeanalysis= pd.DataFrame([])

In [379]:
import datetime
import socket
import pickle
import pandas as pd
from datetime import datetime

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect(("127.0.0.1", 9090))
for _ in range(0,100): 
    start = datetime.now()
    reqFrame = {"Fn":'29','sFn': '08'}
    from_client = {}
    data = pickle.dumps(reqFrame)
    client.send(data)
    data = client.recv(4096)
    ChaCli=''
    from_client = pickle.loads(data)
    if from_client["Cnf"] == '02' and from_client["Fn"] == '69':
        tempData = Req(from_client)
        from_client = tempData
        if from_client['sFn'] == '02':
            ChaCli = from_client['CCli']
        data = pickle.dumps(from_client)
        client.send(data)
        data = client.recv(4096)
        from_client = pickle.loads(data)
        
    if from_client["Cnf"] == '11' and from_client["Fn"] == '69':
        if from_client['sFn'] == '02':
            RecCert = from_client['ECer']
            if CA.verifyCertificate(RecCert,CAPubKeyC ):
                print("Server Certificate Verified")
                if CA.verifySign(ChaCli,from_client["POOC"],RecCert.subjectPublicKey):
                    print("Certificate Verified with proof of Ownership")
    #     temp = CA.sign(from_client["CS"],clientPriKey)
        tempData = Req(from_client)
        from_client = tempData
        data = pickle.dumps(from_client)
        client.send(data)
        data = client.recv(4096)
        from_client = pickle.loads(data)
    if from_client['Cnf'] == "12" and from_client["Fn"] == '69':
        print("Authentication Completed")
    client.close()
    end = datetime.now()
    elapsetime = (end - start).total_seconds()
    timeanalysis = timeanalysis.append([elapsetime])

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [380]:
print('maximum number of iteration',timeanalysis.values.max())
print('minimum number of iteration',timeanalysis.values.min()) 
print('Average number of iteration', timeanalysis.values.mean())

maximum number of iteration 0.009045
minimum number of iteration 0.001676
Average number of iteration 0.005920549450549452
